In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

##### Create dataframe of Team Names and Team IDs. The TeamID column will be used to join the other data tables into a single dataframe.

In [29]:
teams = pd.read_csv("data/MTeams.csv")
teams.drop(['FirstD1Season', 'LastD1Season'], axis=1, inplace=True)
teams

,TeamID,TeamName
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M
...,...,...
367,1468,Bellarmine
368,1469,Dixie St
369,1470,Tarleton St
370,1471,UC San Diego


#### Import data table that shows the ranking of each team according to various ranking systems at various points in the season. Only RankingDayNum = 133 will be used, which corresponds to the day before the tournament begins. The imported data includes a column called SystemName, which includes all NCAA college basketball ranking systems.

In [30]:
rankings = pd.read_csv("data/MMasseyOrdinals.csv")
rankings = rankings[rankings['RankingDayNum'] == 133]
#rankings

#### Merge the team table with the rankings table to get the team name with the rankings

In [31]:
team_data = teams.merge(rankings, how='inner', on='TeamID')

#### Import data table that shows the seed number of each team of each tournament. Remove all leading or trailing letters (which denote conference) from the Seed column and convert to integer

In [32]:
seeds = pd.read_csv("data/MNCAATourneySeeds.csv")
seeds.dropna()
seeds['Seed'] = seeds['Seed'].str[1:]
seeds['Seed'] = seeds['Seed'].str.lstrip('0')
seeds['Seed'] = seeds['Seed'].str.rstrip('abcd')
seeds['Seed'] = seeds['Seed'].astype(int)
seeds

,Season,Seed,TeamID
0,1985,1,1207
1,1985,2,1210
2,1985,3,1228
3,1985,4,1260
4,1985,5,1374
...,...,...,...
2349,2021,12,1457
2350,2021,13,1317
2351,2021,14,1159
2352,2021,15,1331


#### Merge team data table with seeds table on TeamID and Season. This will eliminate all of the rows of teams that did not make the NCAA tournament in a given year.

In [33]:
team_data = seeds.merge(team_data, how='inner', on=['TeamID', 'Season'])

#### Import data that includes every regular season game, which includes the TeamID (WTeamID, LTeamID) and Score (WScore and LScore) of the winning and losing team of each game. The total count that each team won and loss will be calculated for each season. This is used to get regular season win-loss record.

In [34]:
record = pd.read_csv("data/MRegularSeasonCompactResults.csv")
record = record.drop(['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], axis=1)
wins= record.groupby(['Season', 'WTeamID']).count()
wins = wins.rename(columns={'WTeamID':'TeamID', 'LTeamID': 'Wins'})
losses = record.groupby(['Season', 'LTeamID']).count()
losses = losses.rename(columns={'LTeamID':'TeamID', 'WTeamID': 'Losses'})
wins=wins['Wins']
losses=losses['Losses']

#### Merge number of wins and number of losses to the main team data df. Merge on TeamID and Season.

In [35]:
team_data = pd.merge(team_data, wins, left_on=['TeamID', 'Season'], right_on=['WTeamID', 'Season'])
team_data = pd.merge(team_data, losses, left_on=['TeamID', 'Season'], right_on=['LTeamID', 'Season'])

#### Create Win% Column

In [36]:
team_data["Win%"]=round(team_data["Wins"]/(team_data["Wins"]+team_data["Losses"])*100,4)

#### Import data, which includes every tournament game, which includes the TeamID (WTeamID, LTeamID) and Score (WScore and LScore) of the winning and losing team of each game. The total count that each team won will be calculated for each season. The number of wins is the target feature and will be added to the main team data dataframe.

In [37]:
tourney= pd.read_csv("data/MNCAATourneyCompactResults.csv")
tourney = tourney.drop(['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], axis=1)
t_wins= tourney.groupby(['Season', 'WTeamID']).count()
t_wins = t_wins.rename(columns={'WTeamID':'TeamID', 'LTeamID': 'T_Wins'})
t_wins=t_wins['T_Wins']

#### Merge number of tournament wins to df. This will be the target feature

In [38]:
team_data = pd.merge(team_data, t_wins, how='left', left_on=['TeamID', 'Season'], right_on=['WTeamID', 'Season'])


#### Replace NaN with 0's for teams without a win (Teams that lost in the first round)

In [39]:
team_data["T_Wins"] = team_data["T_Wins"].replace(np.nan, 0)

#### Export team data dataframe to csv. More features will be added to this dataframe in another notebook.

In [40]:
team_data.to_csv('team_data.csv')

In [41]:
team_data

,Season,Seed,TeamID,TeamName,RankingDayNum,SystemName,OrdinalRank,Wins,Losses,Win%,T_Wins
0,2003,1,1328,Oklahoma,133,AP,3,24,6,80.0000,3.0
1,2003,1,1328,Oklahoma,133,ARG,5,24,6,80.0000,3.0
2,2003,1,1328,Oklahoma,133,BIH,5,24,6,80.0000,3.0
3,2003,1,1328,Oklahoma,133,BOB,3,24,6,80.0000,3.0
4,2003,1,1328,Oklahoma,133,BRZ,7,24,6,80.0000,3.0
...,...,...,...,...,...,...,...,...,...,...,...
61434,2021,16,1216,Hartford,133,TRP,194,15,8,65.2174,0.0
61435,2021,16,1216,Hartford,133,WIL,190,15,8,65.2174,0.0
61436,2021,16,1216,Hartford,133,WLK,181,15,8,65.2174,0.0
61437,2021,16,1216,Hartford,133,WMV,167,15,8,65.2174,0.0
